In [1]:
# Import Libs
from dotenv import load_dotenv
from sqlalchemy import create_engine

import pandas as pd
import os 

# Load .env
load_dotenv()

True

In [2]:
def get_engagement_data():
    # Load data
    df_engagement = pd.read_excel('Content_2024-03-06_2025-03-05_ThanaelButewicz.xlsx',sheet_name='Engajamento')

    # Rename columns
    df_engagement.rename(columns = {
        'Data':'referencia',
        'Impressões':'impressoes',
        'Engajamento':'engajamento'
    }, inplace = True)

    return df_engagement

In [3]:
def get_follower_data():
    # Load data
    df_followers = pd.read_excel('linkedin_export_data.xlsx',sheet_name='Seguidores')

    # Ajust rowsz
    df_followers = df_followers.iloc[2:].copy()

    # Rename columns
    df_followers.columns = ['referencia','seguidores']

    # Transform "seguidores" to INTEGER format
    df_followers['seguidores'] = df_followers['seguidores'].astype(int)

    return df_followers

In [4]:
def insert_into_database(df):
    # Open the connection with Database
    connection = create_engine(os.environ["string_connection"])

    # Get most recent date in Database
    database_most_recent_date = pd.read_sql('select max(referencia) from linkedin_metricas_diarias', connection)['max'].iloc[0]

    # Filter to get only new data 
    df = df[df['referencia'] > database_most_recent_date]

    # Insert into Database
    df.to_sql('linkedin_metricas_diarias', connection, if_exists = 'append', index = False)

    print(f'\n\nInserido {df.shape[0]} dados em linkedin_metricas_diarias\n\n')
    
    # Close connection
    connection.dispose()

In [5]:
def main():
    # Engagement data
    df_engagement = get_engagement_data()

    # Followers data
    df_followers = get_follower_data()

    # Merge dataframes
    df = pd.merge(
        left = df_engagement,
        right = df_followers,
        on = 'referencia'
    )

    # Transform referencia to DATETIME
    df['referencia'] = pd.to_datetime(df['referencia'],format='%d/%m/%Y')

    # New column 
    df.insert(1,'usuario','thanael')
    
    # Insert data into Database
    insert_into_database(df)


In [6]:
main()

C:\Users\thanael.butewicz\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\thanael.butewicz\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")




Inserido 0 dados em linkedin_metricas_diarias


